In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(loglevel="INFO")
mapdl.clear()
mapdl.prep7()

# Material properties -- Concrete
mapdl.units("SI")  # Use SI units (meters, kilograms, seconds, Kelvin)

# Density
mapdl.mp("DENS", 1, 2400)  # Density in kg/m³

# Define temperature-dependent material properties
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)

# Specific Heat (J/kg*K)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)

# Thermal Conductivity (W/m*K)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# Geometry Parameters
od_media = 0.254  # Outer diameter of media (m)
od_hx = 0.0508  # Outer diameter of HX (m)
t_hx = 0.00635  # Thickness of HX (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of HX (m)
length = 0.508  # Length of the cylinder (m)
num_radial = 5              # Radial divisions
num_circ = 12               # Circumferential divisions

# 1. Create keypoints for the circles
# Center keypoint
k0 = mapdl.k("", 0, 0, 0)

# Inner circle keypoints
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (360/num_circ)
    x_inner = (id_hx/2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx/2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media/2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media/2) * np.sin(np.deg2rad(angle))

    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# 2. Create circular arcs
inner_arcs = []
outer_arcs = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    inner_arcs.append(mapdl.l(inner_kps[i], inner_kps[next_i]))
    outer_arcs.append(mapdl.l(outer_kps[i], outer_kps[next_i]))

# 3. Create radial lines
radial_lines = []
for i in range(num_circ):
    radial_lines.append(mapdl.l(inner_kps[i], outer_kps[i]))

# 4. Create areas for each sector
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    # Lines forming the sector: inner arc, radial line, outer arc, next radial line
    lines = [
        inner_arcs[i],
        radial_lines[next_i],  # Note the order - right side first
        outer_arcs[i],
        radial_lines[i]
    ]
    areas.append(mapdl.al(*lines))

# 5. Set element type for 3D thermal analysis
mapdl.et(1, "SOLID70")

# 6. Extrude each 2D area to create 3D volumes
volumes = []
for area in areas:
    volumes.append(mapdl.vext(area, dz=length))

# 7. Set mesh size before meshing volumes
mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)  # Radial divisions

# set z (new)
num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)

mapdl.allsel()
mapdl.vmesh("ALL")


# 8. Visualize 3D mesh
mapdl.eplot(vtk=True, show_edges=True, line_width=1)

In [ ]:
mapdl.finish()
mapdl.slashsolu()

# Set Up Transient Analysis
mapdl.antype(4)  # Transient thermal analysis
mapdl.trnopt("FULL")  # Full transient solution
mapdl.kbc(0)  # Ramped loading (smooth boundary conditions)
mapdl.timint("ON")  # Enable time integration

# Define initial condition (uniform temperature)
mapdl.ic("ALL", "TEMP", 100)  # Initial temperature of 100°C

# Define time steps
time_steps = np.linspace(0, 14400, 25)  # 0 to 14400 seconds in 24 steps

for i, t in enumerate(time_steps):
    if i == 0:
        continue  # skip initial condition step (already set)

    mapdl.time(t)  # Set current time

    # Apply boundary condition (temperature load)
    # Select appropriate nodes (here we're applying to all nodes as an example)
    # In a real case, we might select specific nodes/surfaces, but I don't know where to get it
    mapdl.nsel("S", "LOC", "Z", 0)  # Example: select nodes at Z=0
    mapdl.d("ALL", "TEMP", 500)  # Apply temperature
    mapdl.nsel("ALL")  # Reselect all nodes

    mapdl.autots("ON")  # Automatic time stepping
    mapdl.deltim(600)  # Suggested time step size (can be adjusted by autots)

    mapdl.solve()  # Solve current time step

mapdl.finish()

In [ ]:
mapdl.finish()
mapdl.post1()

In [ ]:
mapdl.set(1)  # 选择第一个时间步结果
mapdl.plnsol("TEMP")  # 显示温度等值图

In [ ]:
mapdl.post1()
mapdl.set(1)  # 第一个时间步
mapdl.post_processing.plot_nodal_temperature(cmap="inferno")


In [ ]:
import os
import pyvista as pv

output_dir = "temp_frames"
os.makedirs(output_dir, exist_ok=True)

mapdl.post1()
num_steps = len(time_steps)

for i in range(1, num_steps):
    mapdl.set(i)
    mesh = mapdl.mesh.grid
    temp = mapdl.post_processing.nodal_temperature()
    mesh.point_data["Temperature"] = temp

    plotter = pv.Plotter(off_screen=True, window_size=(800, 600))
    plotter.add_mesh(mesh, scalars="Temperature", cmap="inferno", show_edges=False)
    plotter.add_scalar_bar(title="Temperature (°C)")


    plotter.view_vector((1, 1, 0.5))

    plotter.show(screenshot=f"{output_dir}/frame_{i:03d}.png")
    print(f"Saved frame {i}")


In [ ]:
import imageio.v2 as imageio

frames = [imageio.imread(f"{output_dir}/frame_{i:03d}.png") for i in range(1, num_steps)]
imageio.mimsave("transient_temperature.gif", frames, duration=0.5)
